In [17]:
import os
from pathlib import Path

import numpy as np
import matplotlib
matplotlib.use('Agg')  # Headless mode (no GUI)
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import ListedColormap

import constants as k
import utils
from calculate_instantaneous_firing_rate import calculate_firing_rates

In [18]:
data_dir = '/Users/rebekahzhang/data/neural_data'
pickle_dir = Path(os.path.join(data_dir, 'session_pickles'))
figure_dir = os.path.join(data_dir, 'figures')

In [19]:
viridis_palette = ListedColormap(plt.cm.viridis(np.linspace(0, 1, 5)))  # generating 5 colors for 4 quantile + missed
colors = [viridis_palette(i) for i in range(5)]
quantile_colors = colors[:4]
missed_color = colors[4]

In [20]:
units_vetted = pd.read_csv(os.path.join(data_dir, 'units_vetted.csv'), index_col=0).sort_values('unit_id')
sessions_vetted = pd.read_csv(os.path.join(data_dir, 'sessions_vetted.csv'), index_col=0).sort_values('num_units')

In [21]:
def get_data_for_debugging(units_vetted=units_vetted, session_id='RZ051_2024-11-19_str', unit_id=20):
    units_by_session = units_vetted.groupby("session_id")
    test_session = units_by_session.get_group(session_id)
    # session_id = test_session.loc[test_session['id'] == unit_id, "session_id"]
    session_fr = test_session.loc[test_session['id'] == unit_id, "session_fr"].iloc[0]
    events, trials, units = utils.get_session_data(session_id, pickle_dir)
    spikes = units[unit_id]
    return events, trials, spikes, session_fr
events, trials, spikes, session_fr = get_data_for_debugging()

### raster plot

In [22]:
def prepare_data_for_raster(events, trials, spikes, sorter):
    sorted_trial_id = trials.sort_values(by=sorter).trial_id.tolist()
    events_raster = events.groupby('trial_id')
    spikes_raster = spikes.groupby('trial_id')
    return events_raster, spikes_raster, sorted_trial_id

def plot_raster(ax, sorted_trial_id, events_raster, spikes_raster, anchor, anchored_periods, show_legend=True):
    ax.axvline(0, color='tab:gray', linestyle='--', alpha=0.5, label=anchor)

    event_colors = {
        'visual': 'g',
        'wait': 'orange', 
        'cons_reward': 'b', 
        'cons_no_reward': 'r'
    }

    for trial_offset, t in enumerate(sorted_trial_id):
        # Plot spikes first
        if t in spikes_raster.groups:
            trial_spikes = spikes_raster.get_group(t)
            relevant_periods = anchored_periods[anchor]
            relevant_spike_times = trial_spikes.loc[trial_spikes.period.isin(relevant_periods), anchor]
            # Plot spikes (black)
            ax.eventplot(
                relevant_spike_times, 
                lineoffsets=trial_offset, 
                color='k', 
                linelengths=0.8, 
                linewidths=0.4
            )
            irrelevant_spike_times = trial_spikes.loc[~trial_spikes.period.isin(relevant_periods), anchor]
            ax.eventplot(
                irrelevant_spike_times, 
                lineoffsets=trial_offset, 
                # color='darkgrey', 
                color='k',
                linelengths=0.8, 
                linewidths=0.4
            )

        # Plot trial events
        if t in events_raster.groups:
            trial_events = events_raster.get_group(t)
            for event_type, color in event_colors.items():
                event_time = trial_events.loc[trial_events['event_type'] == event_type, anchor]
                ax.eventplot(
                    event_time, lineoffsets=trial_offset, color=color, 
                    linelengths=1.0, linewidths=0.8, alpha=1
                )

    # Create legend only for the last raster plot
    if show_legend:
        handles = [plt.Line2D([0], [0], color=c, lw=2, label=label) 
                   for label, c in event_colors.items()]
        ax.legend(handles=handles, loc='center left', bbox_to_anchor=(1, 0.92))

In [23]:
anchor = k.TO_DECISION
sorter = k.WAIT_LENGTH
_, ax = plt.subplots(figsize=(16,9))
# ax.set_xlim([-7, 3.2])
events_raster, spikes_raster, sorted_trial_id = prepare_data_for_raster(events, trials, spikes, sorter)
plot_raster(ax, sorted_trial_id, events_raster, spikes_raster, anchor, k.ANCHORED_PERIODS, show_legend=True)

### histo plot

In [24]:
def prepare_data_for_histogram_by_quantile(trials, spikes, anchor, anchored_periods, sorter):
    # Calculate the aligned start and end time based on the anchor
    if anchor == k.TO_CUE_ON:
        # boundaries: trial start (0) -> cue off time
        trials['aligned_start_time'] = 0 - trials["cue_on_time"]
        trials['aligned_end_time'] = trials['cue_off_time'] - trials["cue_on_time"]
    
    elif anchor == k.TO_CUE_OFF:
        # boundaries: cue on time -> consumption time (or trial_length if consumption is missing)
        trials['aligned_start_time'] = trials['cue_on_time'] - trials["cue_off_time"]
        # Handle missing decision_time
        mask = trials['decision_time'].isna()
        trials.loc[~mask, 'aligned_end_time'] = trials['decision_time'] - trials["cue_off_time"]
        trials.loc[mask, 'aligned_end_time'] = trials['trial_length'] - trials["cue_off_time"]
    
    elif anchor == k.TO_DECISION:
        # boundaries: cue off time -> trial end
        trials['aligned_start_time'] = trials["cue_off_time"] - trials["decision_time"]
        trials['aligned_end_time'] = trials['trial_length'] - trials["decision_time"]
    
    # Calculate quantiles only on non-missed trials
    non_missed_trials = trials[~trials['missed']].copy()
    non_missed_trials['quantile'] = pd.qcut(non_missed_trials[sorter], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    # missed trials will have NaN quantile
    trials = trials.merge(non_missed_trials[['trial_id', 'quantile']], on='trial_id', how='left')

    # get spikes from ancored periods
    relevant_periods = anchored_periods[anchor]
    spikes_histo = spikes.loc[spikes.period.isin(relevant_periods)].copy()
    
    # get trials with spikes
    trials_with_spikes = spikes_histo['trial_id'].unique()
    trials_histo = trials[trials['trial_id'].isin(trials_with_spikes)].copy().sort_values('quantile')
    
    return trials_histo, spikes_histo

In [25]:
def plot_firing_rates_by_quantile(ax, trials, spikes, session_fr, anchor, time_step, sigma, trial_count_mask, show_legend=True):
    """Plots firing rates on given axes with legend on the right."""
    ax.axvline(0, color='tab:gray', linestyle='--', alpha=0.5, label=anchor)
    
    # Calculate rates for all trials
    bin_centers, mean_fr, sem_fr = calculate_firing_rates(
        trials, spikes, anchor, time_step, trial_count_mask, sigma, session_fr
    )

    if len(bin_centers) > 0:
        ax.plot(bin_centers, mean_fr, 'gray', lw=1, alpha=0.6, label='All trials')
        ax.fill_between(bin_centers, mean_fr-sem_fr, mean_fr+sem_fr, color='gray', alpha=0.1)
    
    for i, quantile in enumerate(trials['quantile'].unique()):
        trials_quantile = trials.loc[trials['quantile'] == quantile]
        spikes_quantile = spikes.loc[spikes.trial_id.isin(trials_quantile.trial_id)]
        bin_centers_q, mean_fr_q, sem_fr_q = calculate_firing_rates(
            trials_quantile, spikes_quantile, anchor, time_step, trial_count_mask, sigma, session_fr
            )

        if len(bin_centers_q) > 0:
            ax.plot(bin_centers_q, mean_fr_q, color=quantile_colors[i], lw=1, label=quantile)
            ax.fill_between(bin_centers_q, mean_fr_q-sem_fr_q, mean_fr_q+sem_fr_q, 
                            color=quantile_colors[i], alpha=0.1)
    
    # Missed trials (missed=True)
    if anchor != k.TO_DECISION:
        trials_missed = trials.loc[trials.missed == True]
        spikes_missed = spikes.loc[spikes.trial_id.isin(trials_missed.trial_id)]
        bin_centers_m, mean_fr_m, sem_fr_m = calculate_firing_rates(
            trials_missed, spikes_missed, anchor, time_step, trial_count_mask, sigma, session_fr
        )

        if len(bin_centers_m) > 0:
            ax.plot(bin_centers_m, mean_fr_m, color=missed_color, linestyle='-', lw=1, label='Missed')
            ax.fill_between(bin_centers_m, mean_fr_m-sem_fr_m, mean_fr_m+sem_fr_m, color=missed_color, alpha=0.1)
            
    # Add the firing rate legend to the right of the plot
    if show_legend:
        ax.legend(bbox_to_anchor=(1, 1.05), loc='upper left')

In [26]:
anchor = k.TO_DECISION
sorter = k.WAIT_LENGTH
time_step = 0.1
sigma = 1
trial_count_mask = 5

_, ax = plt.subplots(figsize=(16, 3))
# ax.set_xlim([-7, 3.2])
trials_histo, spikes_histo = prepare_data_for_histogram_by_quantile(trials, spikes, anchor, k.ANCHORED_PERIODS, sorter)
plot_firing_rates_by_quantile(ax, trials_histo, spikes_histo, session_fr, anchor, time_step, sigma, trial_count_mask, show_legend=True)

### combined

In [27]:
def plot_raster_histo(unit_id, events, trials, spikes, session_fr,
                      anchor, anchored_periods, sorter,
                      time_step, sigma, trial_count_mask, 
                      save_fig=False, regenerate=False, figure_folder=figure_dir):
    fig, axes = plt.subplots(2, 1, figsize=(16, 9), sharex='col', gridspec_kw={'height_ratios': [3, 1]})
    plt.subplots_adjust(right=0.85)

    ax_raster = axes[0]
    ax_rate = axes[1]

    events_raster, spikes_raster, sorted_trial_id = prepare_data_for_raster(events, trials, spikes, sorter)
    plot_raster(ax_raster, sorted_trial_id, events_raster, spikes_raster, anchor, anchored_periods)

    trials_histo, spikes_histo = prepare_data_for_histogram_by_quantile(trials, spikes, anchor, anchored_periods, sorter)
    plot_firing_rates_by_quantile(ax_rate, trials_histo, spikes_histo, session_fr, 
                                  anchor, time_step, sigma, trial_count_mask, show_legend=True)

    ax_raster.set_ylabel("Trial")
    ax_rate.set_ylabel("Normalized Firing Rate")
    ax_rate.set_xlabel("Time (s)")

    plt.suptitle(f"{unit_id} aligned {anchor} sorted by {sorter}")
    plt.tight_layout()

    if save_fig:
        fig_path = f'{figure_folder}/{unit_id}.png'
        if regenerate or not os.path.exists(fig_path):
            fig.savefig(fig_path, bbox_inches='tight', dpi=300, format='png')
        fig.clf()
        plt.close(fig)
    else:
        plt.show()
        plt.close(fig)

In [28]:
anchor = k.TO_DECISION
sorter = k.WAIT_LENGTH
plot_raster_histo("test", events, trials, spikes, session_fr,
                      anchor, k.ANCHORED_PERIODS, sorter,
                      time_step, sigma, trial_count_mask, 
                      save_fig=False, figure_folder=figure_dir, regenerate=False)

/var/folders/fv/h96g2bdx42l2htl2pxpvr84h0000gn/T/ipykernel_21619/3880973329.py:32: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


# Plot raster histo per quantile for each anchor sorter pair

loop through each anchor and sorter combo

In [29]:
time_step = 0.1
sigma = 2
trial_count_mask = 5
save_fig = True
regenerate = False

In [30]:
anchor_sorter_pairs = [
    (k.TO_CUE_ON, k.BACKGROUND_LENGTH),
    (k.TO_CUE_OFF, k.WAIT_LENGTH),
    (k.TO_DECISION, k.WAIT_LENGTH)
]

In [31]:
units_short= units_vetted.head(5)

In [32]:
failed_units = []
plotted = 0
total_plots = len(anchor_sorter_pairs) * len(units_vetted)
units_grouped = units_vetted.groupby("session_id")

for anchor, sorter in anchor_sorter_pairs:
    print(f"plotting anchor {anchor} sorted by {sorter}")
    
    for session_id, session_units in units_grouped:
        print(f"plotting {session_id}")
        print(f"{round((plotted/total_plots*100), 1)} % done")
        
        events, trials, units = utils.get_session_data(session_id, pickle_dir)
        
        region = session_units['region'].iloc[0]
        figure_folder = os.path.join(figure_dir, f"raster_histo_{anchor}_by_{sorter}", region)
        os.makedirs(figure_folder, exist_ok=True)
        
        for i, (_, unit_info) in enumerate(session_units.iterrows()):
            unit_id = unit_info['unit_id']
            session_fr = unit_info['session_fr']
            spikes = units[i]
            figure_path = os.path.join(figure_folder, f"{unit_id}.png")
            
            if os.path.exists(figure_path) and not regenerate:
                plotted += 1
                continue
                
            try:
                # Plot the figure
                fig = plot_raster_histo(
                    unit_id, events, trials, spikes, session_fr,
                    anchor, k.ANCHORED_PERIODS, sorter,
                    time_step, sigma, trial_count_mask,
                    save_fig, regenerate, figure_folder=figure_folder
                )
                plt.close(fig) 
                
            except Exception as e:
                failed_units.append({'unit_id': unit_id, 'error': str(e)})
                plt.close('all')
            plotted += 1

# Print summary
if failed_units:
    print("\nFailed units:")
    display(pd.DataFrame(failed_units))
    print(f"\nTotal failed: {len(failed_units)}/{len(units_vetted)}")
else:
    print("All units processed successfully!")

plotting anchor to_cue_on sorted by background_length
plotting RZ034_2024-07-13_str
0.0 % done
plotting RZ034_2024-07-14_str
0.7 % done
plotting RZ036_2024-07-12_str
1.1 % done
plotting RZ036_2024-07-12_v1
1.7 % done
plotting RZ036_2024-07-13_str
1.9 % done
plotting RZ036_2024-07-14_str
2.3 % done
plotting RZ037_2024-07-16_str
2.8 % done
plotting RZ037_2024-07-17_str
3.5 % done
plotting RZ037_2024-07-18_str
4.2 % done
plotting RZ037_2024-07-18_v1
4.4 % done
plotting RZ038_2024-07-17_v1
4.6 % done
plotting RZ038_2024-07-18_str
4.6 % done
plotting RZ038_2024-07-19_str
5.1 % done
plotting RZ039_2024-07-17_str
5.4 % done
plotting RZ047_2024-11-19_str
5.6 % done
plotting RZ047_2024-11-20_v1
6.1 % done
plotting RZ047_2024-11-21_str
6.6 % done
plotting RZ047_2024-11-21_v1
6.8 % done
plotting RZ047_2024-11-22_str
6.9 % done
plotting RZ047_2024-11-22_v1
7.0 % done
plotting RZ049_2024-10-30_str
7.2 % done
plotting RZ049_2024-10-31_str
7.3 % done
plotting RZ049_2024-11-01_str
7.9 % done
plotting 